# <center>Otimização de tabela do Campeonato Brasileiro de Futebol usando Algoritmos Genéticos</center>

## 1. Introdução

O Campeonato Brasileiro de Futebol é o principal torneio do esporte mais popular do país. Com 20 times, tem representantes de quase todas as regiões do Brasil (região Norte já esteve presente em edições passadas). Como o Brasil é um país de tamanho continental, o torneio envolve viagens longas, principalmente para os times do Nordeste.<br> 

A partir do conjunto de premissas apresentadas adiante, vamos utilizar Algoritmos Genéticos para propor uma tabela que minimize o impacto dessas longas viagens, embora seja claro que haverá times que estão geograficamente mais distantes e não há como evitar diferenças. Propomos três métricas para essa otimização, todas utilizando as  distâncias viajadas por cada time.<br>

Esse é um tipo de problema conhecido como TTP (*Traveling Tournament Problem*), Problema de Torneio Itinerante. Existe um conjunto gigante de possíveis ordenamentos e é bastante complicado obter o resultado ótimo, principalmente com um número grande de participantes. 

## 2. Dados disponíveis

Temos duas entradas importantes para poder realizar esse estudo.
1. A lista de times participantes do campeonato;
2. A tabela do campeonato de 2020.

**Vamos importar as bibliotecas necessárias para apresentar os dados.**

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

%matplotlib inline

### 2.1. Lista de participantes

Precisamos da lista com o nome dos 20 participantes do campeonato, junto com a sua cidade de origem. Como estamos interessados na distância viajada por cada equipe, vamos pegar as coordenadas (latitude e longitude) do ponto central de cada cidade para calcular o percurso entre as partidas.

In [6]:
lista_times_20=pd.read_csv('times.csv')
lista_times_20.head()

,Time,Cidade
0,Atlético Goianiense,Goiânia
1,Atlético Mineiro,Belo Horizonte
2,Athletico Paranaense,Curitiba
3,Bahia,Salvador
4,Botafogo,Rio de Janeiro


Montaremos posteriormente uma tabela 20x20 com a distância entre as cidades de cada time.

### 2.2. Tabela de 2020

Pegamos como ponto de partida a programação do campeonato de 2020, onde temos a ordem das partidas para cada um dos 20 times. O torneio é realizado em duas fases, onde a segunda parte repete os jogos da primeira, com mando de campo invertido. Com isso, só precisamos nos preocupar com a metade dos jogos.

Para o nosso problema, organizamos a montagem das primeiras 19 rodadas em duas tabelas, uma com a sequência de times enfrentados e outra com a sequência de mandos de campo (*casa/fora*).

#### 2.2.1 Sequência de jogos

In [7]:
enfrentamentos_20=pd.read_csv('enfrentamentos.csv')
enfrentamentos_20.head()

,Time,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,Rodada 10,Rodada 11,Rodada 12,Rodada 13,Rodada 14,Rodada 15,Rodada 16,Rodada 17,Rodada 18,Rodada 19
0,Athlético Paranaense,Fortaleza,Goiás,Santos,Palmeiras,Fluminense,Atlético Mineiro,Red Bull Bragantino,Vasco da Gama,Botafogo,Coritiba,São Paulo,Bahia,Flamengo,Ceará,Internacional,Corinthians,Atlético Goianiense,Grêmio,Sport
1,Atlético Goianiense,Corinthians,Flamengo,Sport,Internacional,Goiás,Ceará,Fluminense,Grêmio,Vasco da Gama,Bahia,Atlético Mineiro,Botafogo,Fortaleza,São Paulo,Red Bull Bragantino,Santos,Athlético Paranaense,Palmeiras,Coritiba
2,Atlético Mineiro,Flamengo,Corinthians,Ceará,Botafogo,Internacional,Athlético Paranaense,São Paulo,Coritiba,Santos,Red Bull Bragantino,Atlético Goianiense,Grêmio,Vasco da Gama,Fortaleza,Goiás,Fluminense,Bahia,Sport,Palmeiras
3,Bahia,Botafogo,Coritiba,Red Bull Bragantino,São Paulo,Ceará,Palmeiras,Flamengo,Internacional,Grêmio,Atlético Goianiense,Corinthians,Athlético Paranaense,Sport,Vasco da Gama,Fluminense,Goiás,Atlético Mineiro,Fortaleza,Santos
4,Botafogo,Bahia,Red Bull Bragantino,Fortaleza,Atlético Mineiro,Flamengo,Internacional,Coritiba,Corinthians,Athlético Paranaense,Vasco da Gama,Santos,Atlético Goianiense,Fluminense,Palmeiras,Sport,Grêmio,Goiás,São Paulo,Ceará


#### 2.2.2 Mando de campo

In [9]:
mando_20=pd.read_csv('mando.csv')
mando_20.head()

,Time,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,Rodada 10,Rodada 11,Rodada 12,Rodada 13,Rodada 14,Rodada 15,Rodada 16,Rodada 17,Rodada 18,Rodada 19
0,Athlético Paranaense,f,c,f,c,c,f,c,f,c,c,f,c,f,c,f,c,f,c,f
1,Atlético Goianiense,f,c,c,f,f,c,f,c,f,f,c,c,f,f,c,f,c,c,f
2,Atlético Mineiro,f,c,c,f,f,c,c,f,f,c,f,c,c,f,c,c,f,c,f
3,Bahia,f,c,c,f,f,c,c,f,c,c,f,f,c,c,f,f,c,c,f
4,Botafogo,c,f,f,c,f,c,c,f,f,c,c,f,c,c,f,f,c,f,c


## 3. Metodologia

Esse problema é complexo, pois envolve muitas combinações possíveis. Para o Campeonato Brasileiro, o mais provável é que a tabela seja montada de maneira artesanal, respeitando apenas uma premissa:
1. Nenhum time pode jogar mais do que duas partidas seguidas em casa ou fora.


Para o nosso exercício, vamos trabalhar com uma versão *express* do Campeonato Brasileiro. O campeonato continua com as suas 38 rodadas, mas temos mais uma premissa em nosso modelo: são duas partidas por semana.

2. O time não retorna para sua cidade após cada partida, indo diretamente para a sede da próxima partida.

Isso só é relevante para a avaliação dos resultados do algoritmo genético, não influenciando na tabela de jogos.

Temos 4 tabelas em nosso conjunto de dados:
1. lista de times
2. lista de confrontos
3. lista de mandos de campo
4. distância entre sedes

A última tabela é apenas utilizada na função de avaliação. Vamos mexer nas três outras tabelas para podermos aplicar nosso algoritmo genético.

Primeiramente, vamos generalizar as tabelas, substituindo os nomes dos times por uma posição da equipe. Por exemplo, Athlético Paranaense será subtituido por Time01, Atlético Goianiense como Time02, etc. Dessa forma, temos três novas tabelas genéricas.

In [22]:
generico=['Time01','Time02','Time03','Time04','Time05','Time06','Time07','Time08','Time09','Time10','Time11','Time12','Time13','Time14','Time15','Time16','Time17','Time18','Time19','Time20']

In [23]:
lista=lista_times_20['Time'].tolist()
lista_times = lista_times_20.replace(lista,generico)
lista_times.drop(columns=['Cidade'],inplace=True)
lista_times.head()

,Time
0,Time01
1,Time02
2,Time03
3,Time04
4,Time05


In [32]:
lista = enfrentamentos_20['Time'].tolist()
enfrentamentos = enfrentamentos_20.replace(lista,generico)
enfrentamentos.head()

,Time,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,Rodada 10,Rodada 11,Rodada 12,Rodada 13,Rodada 14,Rodada 15,Rodada 16,Rodada 17,Rodada 18,Rodada 19
0,Time01,Time11,Time12,Time17,Time15,Time10,Time03,Time16,Time20,Time05,Time08,Time18,Time04,Time09,Time06,Time14,Time07,Time02,Time13,Time19
1,Time02,Time07,Time09,Time19,Time14,Time12,Time06,Time10,Time13,Time20,Time04,Time03,Time05,Time11,Time18,Time16,Time17,Time01,Time15,Time08
2,Time03,Time09,Time07,Time06,Time05,Time14,Time01,Time18,Time08,Time17,Time16,Time02,Time13,Time20,Time11,Time12,Time10,Time04,Time19,Time15
3,Time04,Time05,Time08,Time16,Time18,Time06,Time15,Time09,Time14,Time13,Time02,Time07,Time01,Time19,Time20,Time10,Time12,Time03,Time11,Time17
4,Time05,Time04,Time16,Time11,Time03,Time09,Time14,Time08,Time07,Time01,Time20,Time17,Time02,Time10,Time15,Time19,Time13,Time12,Time18,Time06


In [33]:
lista =mando_20['Time'].tolist()
mando = mando_20.replace(lista,generico)
mando = mando.replace(['c','f'],[0,1])
mando.head()

,Time,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,Rodada 10,Rodada 11,Rodada 12,Rodada 13,Rodada 14,Rodada 15,Rodada 16,Rodada 17,Rodada 18,Rodada 19
0,Time01,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1
1,Time02,1,0,0,1,1,0,1,0,1,1,0,0,1,1,0,1,0,0,1
2,Time03,1,0,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,0,1
3,Time04,1,0,0,1,1,0,0,1,0,0,1,1,0,0,1,1,0,0,1
4,Time05,0,1,1,0,1,0,0,1,1,0,0,1,0,0,1,1,0,1,0


De novo, esse problema é muito complexo. Precisaríamos trabalhar com as três tabelas ao mesmo tempo para chegar na solução ótima. Como não temos a menor ideia de como fazer isso, vamos atacar o problema em três frentes.

### 3.1. Lista de times

Vamos tentar otimizar a ordem dos times, mantendo as duas outras tabelas fixas. Ou seja, a questão é definir qual equipe é o Time01, qual é o Time02, até o Time20. 

In [36]:
print('Existem',math.factorial(20),'possibilidades para esse arranjo de times.')

Existem 2432902008176640000 possibilidades para esse arranjo de times.


Obviamente esse é um problema que teria um alto custo computacional caso fosse necessário varrer todas as possibilidades.

Em nosso algoritmo genético, o nosso gene é um vetor com números de 1 a 20, correlacionando a lista de equipes participantes com as etiquetas Time01, Time02,..., Time20. A mutação proposta em cada geração é a mudança de posição entre duas equipes.

### 3.2. Mandos de campo

Vamos tentar otimizar os mandos de campo, mantendo as duas outras tabelas fixas. Ou seja, a questão é definir a sequência de jogos em casa ou fora para cada equipe, mas devemos respeitar a premissa de não ter mais de duas partidas em casa ou fora em sequência.

Em nosso algoritmo genético, o nosso gene é a matriz 20x19, onde cada linha é um time e cada coluna é uma rodada.  A mutação proposta em cada geração é a escolha aleatória de uma partida na tabela de enfrentamentos e a inversão do mando de campo na outra tabela. Ela deve respeitar a premissa, ou seja, uma mutação só será aceita se não criar um caso onde haja 3 partidas seguidas em casa ou fora para os dois times envolvidos.

### 3.3. Ordem de enfrentamentos

Vamos tentar otimizar a ordem das rodadas, mantendo as duas outras tabelas fixas. Ou seja, a questão é definir a sequência de times que cada um vai enfrentar. De novo, precisamos respeitar a premissa de não ter mais de duas partidas em casa ou fora em sequência.

Em nosso algoritmo genético, o nosso gene é um vetor com números de 1 a 19. A mutação proposta em cada geração é a mudança de posição entre duas rodadas. Ela deve respeitar a premissa, ou seja, uma mutação só será aceita se não criar um caso onde haja 3 partidas seguidas em casa ou fora para os dois times envolvidos.

## 4. Métrica de avaliação

O parâmetro que usaremos para avaliação é a distância percorrida por cada equipe. Por exemplo, para o Time01, é a distância entre a sua cidade e o local da sua primeira partida, mais a distância entre o local da primeira partida e o local da segunda, e assim por diante. <br>
Lembrando, nossas duas premissas são:
1. Nenhum time pode jogar mais do que duas partidas seguidas em casa ou fora.
2. O time não retorna para sua cidade após cada partida, indo diretamente para a sede da próxima partida.

Independente da métrica que usarmos para avaliar os genes, o vetor com a distância percorrida por cada equipe será o dado fundamental.

Temos 3 opções para uma métrica:
1. minimização da soma total de distâncias;
2. minimização da diferença entre maior e menor distância percorrida;
3. minimização do desvio padrão da distribuição de distâncias.

Para o nosso exercício, queremos minimizar o impacto de longas viagens no desempenho dos times. Com isso em mente, vamos nos concentrar na opção 3, minimizar o desvio padrão da distribuição das distâncias. Podemos ter *outliers* com distâncias muito grandes ou muito pequenas, mas garantimos que a maior parte das equipes fique próximo da média.

## 5. Atualização para 2021

Vamos carregar a lista de participantes para o Campeonato de 2021, com as cidades de cada equipe. 

In [82]:
lista_times_21=pd.read_csv('times_21.csv')
lista_times_21.head()

,Time,Cidade
0,América,Belo Horizonte
1,Athlético Paranaense,Curitiba
2,Atlético Goianiense,Goiânia
3,Atlético Mineiro,Belo Horizonte
4,Bahia,Salvador


Vamos usar uma biblioteca que retorna as coordenadas (Latitude e Longitude) de cada cidade e vamos criar uma tabela com a distância entre as cidades. Cada linha é um dos times, assim como as colunas.

Importando as bibliotecas necessárias.

In [83]:
!pip3 install geopy
from geopy.geocoders import Nominatim
import time
app = Nominatim(user_agent="gabrasileirao")

Definindo a função que busca as coordenadas.

In [84]:
def get_location_by_address(address):
    """This function returns a location as raw from an address
    will repeat until success"""
    time.sleep(1)
    try:
        return app.geocode(address).raw
    except:
        return get_location_by_address(address)

Buscando as coordenadas para cada cidade na lista.

In [85]:
lista_times_21['Latitude']=0.
lista_times_21['Longitude']=0.
for ii in range(lista_times_21.shape[0]):
    print(ii)
    address = [lista_times_21.iloc[ii,1],'Brazil']
    location = get_location_by_address(address)
    lista_times_21.iloc[ii,-2] = location["lat"]
    lista_times_21.iloc[ii,-1] = location["lon"]
lista_times_21.head()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


,Time,Cidade,Latitude,Longitude
0,América,Belo Horizonte,-19.9227318,-43.9450948
1,Athlético Paranaense,Curitiba,-25.4295963,-49.2712724
2,Atlético Goianiense,Goiânia,-16.680882,-49.2532691
3,Atlético Mineiro,Belo Horizonte,-19.9227318,-43.9450948
4,Bahia,Salvador,-12.9822499,-38.4812772


In [88]:
lista_times_21['Latitude']=lista_times_21['Latitude'].astype(float)
lista_times_21['Longitude']=lista_times_21['Longitude'].astype(float)

Vamos conferir no mapa se está tudo correto.

In [89]:
!pip install folium
import folium # map rendering library

In [90]:
#mapa centrado em Brasília
map_brasil = folium.Map(location=[-10.3333333, -53.2], zoom_start=4)

# add markers to map
for lat, lng, time in zip(lista_times_21['Latitude'], lista_times_21['Longitude'], lista_times_21['Time']):
    label = '{}'.format(time)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brasil)  
    
map_brasil

Temos nossos 20 times espalhados em 14 cidades, em 11 estados. Agora precisamos construir a tabela com as distâncias. De novo, vamos definir uma função que faz esse cálculo.

In [114]:
#função encontrada na internet para calcular a distância entre dois pontos a partir de suas coordenadas
from math import cos, asin, sqrt, pi

def distance(lat1,lon1,lat2,lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) + cos(lat2*p)*(1-cos((lon2-lon1)*p))/2
    if a>1:
        a=1;
    return 12742.*asin(sqrt(a)) 
#12742 é o diâmetro da Terra


Vamos criar a tabela e depois populá-la com as distâncias.

In [115]:
distancias=pd.DataFrame(columns=['Time']+lista_times_21['Time'].tolist())
distancias['Time']=lista_times_21['Time'].tolist()
distancias.head()

,Time,América,Athlético Paranaense,Atlético Goianiense,Atlético Mineiro,Bahia,Bragantino,Ceará,Chapecoense,Corinthians,...,Flamengo,Fluminense,Fortaleza,Grêmio,Internacional,Juventude,Palmeiras,Santos,São Paulo,Sport
0,América,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Athlético Paranaense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Atlético Goianiense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Atlético Mineiro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bahia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Importando a biblioteca que calcula a distância a partir das coordenadas.

In [120]:
!pip install haversine
import haversine as hs


In [121]:
for ii in range(len(lista_times_21)):
    for jj in range(len(lista_times_21)):
        loc1=(lista_times_21.iloc[ii,-2],lista_times_21.iloc[ii,-1])
        loc2=(lista_times_21.iloc[jj,-2],lista_times_21.iloc[jj,-1])
        distancias.iloc[ii,jj+1]=hs.haversine(loc1,loc2)

distancias.head()

,Time,América,Athlético Paranaense,Atlético Goianiense,Atlético Mineiro,Bahia,Bragantino,Ceará,Chapecoense,Corinthians,...,Flamengo,Fluminense,Fortaleza,Grêmio,Internacional,Juventude,Palmeiras,Santos,São Paulo,Sport
0,América,0,820.485,666.215,0,966.725,430.9,1894.08,1189.77,489.691,...,340.896,340.896,1894.08,1341.83,1341.83,1261.11,489.691,512.134,489.691,1640.7
1,Athlético Paranaense,820.485,0,972.816,820.485,1786.64,390.537,2672.15,381.377,338.956,...,675.611,675.611,2672.15,546.903,546.903,456.486,338.956,338.708,338.956,2460.42
2,Atlético Goianiense,666.215,972.816,0,666.215,1228.42,752.725,1856.43,1208.38,811.308,...,937.644,937.644,1856.43,1498.19,1498.19,1402.42,811.308,864.739,811.308,1830
3,Atlético Mineiro,0,820.485,666.215,0,966.725,430.9,1894.08,1189.77,489.691,...,340.896,340.896,1894.08,1341.83,1341.83,1261.11,489.691,512.134,489.691,1640.7
4,Bahia,966.725,1786.64,1228.42,966.725,0,1397.62,1028.76,2150.66,1455.69,...,1211.69,1211.69,1028.76,2305.24,2305.24,2226.57,1455.69,1474.19,1455.69,673.975


In [122]:
distancias=distancias.set_index('Time')
distancias.head()

,América,Athlético Paranaense,Atlético Goianiense,Atlético Mineiro,Bahia,Bragantino,Ceará,Chapecoense,Corinthians,Cuiabá,Flamengo,Fluminense,Fortaleza,Grêmio,Internacional,Juventude,Palmeiras,Santos,São Paulo,Sport
Time,,,,,,,,,,,,,,,,,,,,
América,0,820.485,666.215,0,966.725,430.9,1894.08,1189.77,489.691,1373.35,340.896,340.896,1894.08,1341.83,1341.83,1261.11,489.691,512.134,489.691,1640.7
Athlético Paranaense,820.485,0,972.816,820.485,1786.64,390.537,2672.15,381.377,338.956,1303.37,675.611,675.611,2672.15,546.903,546.903,456.486,338.956,338.708,338.956,2460.42
Atlético Goianiense,666.215,972.816,0,666.215,1228.42,752.725,1856.43,1208.38,811.308,741.012,937.644,937.644,1856.43,1498.19,1498.19,1402.42,811.308,864.739,811.308,1830
Atlético Mineiro,0,820.485,666.215,0,966.725,430.9,1894.08,1189.77,489.691,1373.35,340.896,340.896,1894.08,1341.83,1341.83,1261.11,489.691,512.134,489.691,1640.7
Bahia,966.725,1786.64,1228.42,966.725,0,1397.62,1028.76,2150.66,1455.69,1919.91,1211.69,1211.69,1028.76,2305.24,2305.24,2226.57,1455.69,1474.19,1455.69,673.975
Bragantino,430.9,390.537,752.725,430.9,1397.62,0,2304.96,765.68,67.2179,1293.34,341.295,341.295,2304.96,914.914,914.914,831.893,67.2179,114.163,67.2179,2071.59
Ceará,1894.08,2672.15,1856.43,1894.08,1028.76,2304.96,0,2998.52,2369.89,2332.11,2191.62,2191.62,0,3215.62,3215.62,3128.01,2369.89,2400.6,2369.89,627.18
Chapecoense,1189.77,381.377,1208.38,1189.77,2150.66,765.68,2998.52,0,718.682,1327.7,1055.41,1055.41,2998.52,353.856,353.856,270.472,718.682,720.022,718.682,2822.66
Corinthians,489.691,338.956,811.308,489.691,1455.69,67.2179,2369.89,718.682,0,1327.75,357.01,357.01,2369.89,852.799,852.799,771.418,0,54.8576,0,2129.6
